# Aula 1 - Parte Prática - Agentes & Ambientes

## Introdução

Nesse primeiro notebook iremos aprender sobre a API do [OpenAI Gym](http://gym.openai.com/) e começaremos a implementar alguns componentes de um agente de RL.
Ao final dessa parte prática teremos implementado o **ciclo de interação Agente-Ambiente** que nos permitirá futuramente treinar o agente e avaliar sua performance.

### Objetivos:

- Relacionar os conceitos de Processos de Decisão Markovianos (MDPs) com os atributos e métodos de um ambiente definido com o OpenAI Gym;
- Familiarizar-se com os componentes básicos de um agente de RL;
- Implementar o ciclo de interação Agente-Ambiente; e
- Implementar um primeiro agente aleatório e avaliar sua performance

### Imports

> **Atenção:** não se esqueça de executar todos os `imports` necessários antes prosseguir com o tutorial. 

In [44]:
import abc
import random

import gym

from utils.viz import plot_action_distribution, plot_episode_total_rewards

## 1. Ambientes no OpenAI Gym

Um ambiente no OpenAI Gym encapsula um simulador com o qual um agente pode interagir. Nesse simulador, a cada instante $t$ o agente deve escolhar uma ação $\mathbf{a}_t \in \mathcal{A}$ a ser executada. Ao receber essa ação, o ambiente tem seu estado $\mathbf{s}_t \in \mathcal{S}$ alterado para outro estado $\mathbf{s}_{t+1} \in \mathcal{S}$ e devolve para o agente uma observação (que pode ou não corresponder ao estado) e uma recompensa/punição $r_t \in \mathbb{R}$.

O pacote Gym conta com inúmeros [ambientes pré-programados](http://gym.openai.com/envs/) e prontos para serem usados para testar algoritmos de RL. Nessa parte prática, começaremos a explorar alguns ambientes mais simples a fim de nos familiarizarmos com os principais conceitos de modelagem da biblioteca Gym.

Para carregar um ambiente disponível no Gym basta chamar a função `gym.make` passando como argumento o identificador do ambiente:

In [45]:
env = gym.make("MountainCar-v0")

> **Atenção**: antes de usar um ambiente do Gym com o qual você não está familiarizado pode ser útil ler a sua documentação online.
> Para o `MountainCar-v0` acesse o link [http://gym.openai.com/envs/MountainCar-v0/](http://gym.openai.com/envs/MountainCar-v0/).

Informações adicionais sobre o ambiente/simulador podem ser obtidas accessando o atributo `env.spec`. Embora não seja obrigatório, muitos ambientes do OpenAI Gym definem  metadados importantes relacionados à tarefa de RL. Em particular, `env.spec.max_episode_steps` define o número máximo de passos de decisão que um agente pode tomar em um episódio (i.e., o tamanho máximo de uma trajetória) e `env.spec.reward_threshold` define o valor mínimo de retorno (i.e., recompensa total) de um episódio para o qual a tarefa é considerada resolvida.

In [46]:
print(f"MAX_EPISODE_STEPS = {env.spec.max_episode_steps}")
print(f"REWARD THRESHOLD = {env.spec.reward_threshold}")

MAX_EPISODE_STEPS = 200
REWARD THRESHOLD = -110.0


### 1.1. Espaço de estados e ações

Todo agente de RL deve conhecer quais ações pode tomar no ambiente e também quais as características das variáveis de observações que aquele ambiente lhe disponibiliza. Para acessar o espaço de estados e ações, um ambiente do gym disponibiliza os atributos `env.observation_space` e `env.action_space`, respectivamente.

Note que essas informações serão importantes futuramente na definição das entradas e saídas das redes neurais artificiais que utilizaremos para representar a política $\pi_\theta$ do agente e também na criação de outros modelos.   

Todo `observation_space` tem associado seu tipo numérico (e.g., int, float,...) e as dimensões de uma observação. Além disso, é possível saber se o valor das variáveis observação são limitadas ou não, e se forem limitadas qual o valor mínimo e máximo.

In [47]:
obs_space = env.observation_space

print(obs_space.dtype)
print(obs_space.shape)
print(obs_space.bounded_above, obs_space.low)
print(obs_space.bounded_below, obs_space.high)

float32
(2,)
[ True  True] [-1.2  -0.07]
[ True  True] [0.6  0.07]


Para o `MountainCarContinuous-v0` note que uma observação correponde a um vetor de números reais de tamanho 2 (i.e., um par de valores em ponto flutuante). Note que a primeira componente do vetor é limitada entre -1.2 e 0.6, e a segunda componente entre -0.07 e 0.07. 

Analogamente, todo `action_space` tem associado seu tipo numérico e as dimensões de uma ação:

In [48]:
action_space = env.action_space

print(action_space.dtype)
print(action_space.shape)
print(action_space.n)

int64
()
3


Note que para o `MountainCar-v0` o agente deverá escolher ações discretas (i.e., representadas por números inteiros). Observe também que o `shape==()` indica que uma ação é dada por um único escalar (e não um vetor como no caso do `observation_space`. Para acessar o número de possíveis valores da ação basta acessar o atributo `action_space.n`.

> **Atenção**: em algumas situações pode ser interessante ter acesso à amostras de observações e ações. Você pode usar os métodos `env.observation_space.sample()` e `env.action_space.sample()` para gerar aleatoriamente observações e ações direto do ambiente `env`. 

In [49]:
obs_samples = [env.observation_space.sample() for _ in range(3)]
print(obs_samples)

[array([-0.26494658,  0.01483747], dtype=float32), array([-0.12745877,  0.01524209], dtype=float32), array([-0.69368374, -0.02006871], dtype=float32)]


In [50]:
action_samples = [env.action_space.sample() for _ in range(5)]
print(action_samples)

[1, 1, 0, 0, 2]


### 1.2 Interface do Gym: métodos reset, step, render e close

Um objeto `env` do Gym fornece 4 métodos principais para interagir com o simulador: 
1. `reset` permite re-inicialiar o simulador para um de seus estados iniciais;
2. `step` se encarrega de executar uma ação no ambiente;
3. `render` visualiza graficamente o estado do agente; e
4. `close` libera recursos utilizados na simulação (por exemplo fecha a janela de visualização).

In [51]:
for i in range(5):
    obs = env.reset()
    print(f"obs {i} = {obs}")

obs 0 = [-0.58980679  0.        ]
obs 1 = [-0.50968261  0.        ]
obs 2 = [-0.48926999  0.        ]
obs 3 = [-0.46957288  0.        ]
obs 4 = [-0.43670477  0.        ]


Note que a observação do estado inicial muda conforme o método `.reset()` é chamado. Isso se deve ao fato de que o estado inicial é definido como uma variável aleatória regida por uma distribuição inicial.

In [52]:
obs = env.reset()

for i in range(5):
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)
    print(f"transition {i} = ({action}, {obs}, {reward})")

transition 0 = (1, [-5.06570580e-01 -1.28619302e-04], -1.0)
transition 1 = (1, [-5.06826856e-01 -2.56275226e-04], -1.0)
transition 2 = (1, [-5.07208867e-01 -3.82011557e-04], -1.0)
transition 3 = (0, [-0.50871375 -0.00150489], -1.0)
transition 4 = (0, [-0.51133024 -0.00261649], -1.0)


> **Atenção**: Você pode se familiarizar com os argumentos e retorno de ambos os métodos através de seus *docstrings* acessados via `??`.

In [53]:
env.reset?

Signature: env.reset(**kwargs)
Docstring:
Resets the state of the environment and returns an initial observation.

Returns:
    observation (object): the initial observation.
File:      ~/anaconda3/envs/rlsummer/lib/python3.6/site-packages/gym/wrappers/time_limit.py
Type:      method


In [54]:
env.step?

Signature: env.step(action)
Docstring:
Run one timestep of the environment's dynamics. When end of
episode is reached, you are responsible for calling `reset()`
to reset this environment's state.

Accepts an action and returns a tuple (observation, reward, done, info).

Args:
    action (object): an action provided by the agent

Returns:
    observation (object): agent's observation of the current environment
    reward (float) : amount of reward returned after previous action
    done (bool): whether the episode has ended, in which case further step() calls will return undefined results
    info (dict): contains auxiliary diagnostic information (helpful for debugging, and sometimes learning)
File:      ~/anaconda3/envs/rlsummer/lib/python3.6/site-packages/gym/wrappers/time_limit.py
Type:      method


## 2. Agentes 

A fim de permitir a implementação do **ciclo de interação Agente-Ambiente**, um agente de RL deve ser capaz de escolher uma ação para cada observação recebida do ambiente e aprender (i.e., melhorar sua performance) a partir de suas experiências.

Nesse contexto, na classe abstrata `RLAgent` definimos a interface geral de um agente de RL. 

> **Atenção**: Familiarize-se com essa classe; todos os agentes definidos nessa aula e nas próximas deverão especializar (i.e., derivar ou sub-classear) essa interface geral.

In [55]:
class RLAgent:
    """
    Classe abstrata que define a interface básica de um agente RL.

    Args:
        obs_space:     especificação do espaço de observações do ambiente.
        action_space:  especificação do espaço de ações do ambiente.
        config (dict): (opcional) configurações de hiper-parâmetros.
    """
    
    __metaclass__ = abc.ABCMeta

    def __init__(self, obs_space, action_space, config=None):
        self.obs_space = obs_space
        self.action_space = action_space
        self.config = config

    @abc.abstractmethod
    def act(self, obs):
        """
        Escolhe uma ação para ser tomada dada uma observação do ambiente.
        
        Args: 
            obs: observação do ambiente.
        
        Return:
            action: ação válida dentro do espaço de ações.
        """
        raise NotImplementedError

    @abc.abstractmethod
    def observe(self, obs, action, reward, next_obs, done):
        """
        Registra na memória do agente uma transição do ambiente.

        Args:
            obs:            observação do ambiente antes da execução da ação.
            action:         ação escolhida pelo agente.
            reward (float): escalar indicando a recompensa obtida após a execução da ação.
            next_obs:       nova observação recebida do ambiente após a execução da ação.
            done (bool):    True se a nova observação corresponde a um estado terminal, False caso contrário.

        Return:
            None
        """
        raise NotImplementedError

    @abc.abstractmethod
    def learn(self):
        """
        Método de treinamento do agente. A partir das experiências de sua memória,
        o agente aprende um novo comportamento.

        Args: 
            None

        Return:
            None
        """     
        raise NotImplementedError


### 2.1 Definindo um agente aleatório

Antes de finalmente definir o ciclo de interação agente-ambiente, vamos implementar um agente que escolhe ações aleatórias.

O agente `RandomPolicy` tem seu comportamento definido por uma política estocástica dada por uma distribuição uniforme sobre as ações válidas:

$$
\mathbf{a}_t \sim \pi(\cdot|\mathbf{s}) = \mathcal{Uniform}(\{ \mathbf{a} : \mathbf{a} \in \mathcal{A} \})~.
$$

Note que nesse ponto do curso, a implementação do agente aleatório é basicamente ilustrativa. No entanto, como veremos nas aulas seguintes, um agente que implementa uma política aleatória tem duas importantes funções:
1. servir de referência de performance final; e
2. guiar a inicialização de agentes de RL.

Em outras palavras, se um agente de RL após o treinamento não conseguir uma performance significativamente melhor do que aquela do agente aleatório, então muito provavelemente algo não está funcionando como deveria. Além disso, ao garantir que a inicialização de uma política induza um comportamente similar ao de um agente aleatório, não estaremos enviesando a exploração inicial do agente; o que poderia levar muito rapidamente para uma performance sub-ótima.

In [56]:
class RandomPolicy(RLAgent):
    """
    Agente aleatório. Escolhe aleatoriamente uma ação independentemente
    da observação recebida do ambiente.
    
    Args:
        action_space:  especificação do espaço de ações do ambiente.
    """
    
    def __init__(self, observation_space, action_space, config=None):
        super(RandomPolicy, self).__init__(observation_space, action_space, config)

    def act(self, obs):
        """Retorna uma ação aleatória."""
        return self.action_space.sample()

    def observe(self, obs, action, reward, next_obs, done):
        """Ignora transições; i.e., um agente aletório não armazena experiências na memória."""
        pass

    def learn(self):
        """Um agente aletório não aprende; i.e., não melhora seu comportamento."""
        pass


Note que para uma mesma observação o agente `RandomPolicy` retorna diferentes ações a cada chamada:

In [57]:
agent = RandomPolicy(env.observation_space, env.action_space)

obs = env.reset()
print([agent.act(obs) for _ in range(20)])

[0, 2, 2, 2, 1, 0, 0, 0, 0, 0, 2, 2, 1, 0, 0, 0, 1, 0, 1, 1]


In [58]:
plot_action_distribution(agent)

## 3. Ciclo de Interação Agente-Ambiente

Após entender a API do OpenAI Gym e se familiarizar com a interface geral de um agente de RL, estamos pronto para programar o ciclo de interação Agente-Ambiente.

Note que tanto o *treinamento* como a *avaliação de performance* de agentes de RL (baseados em simuladores) dependem da coleta de experiências a fim de estimar uma diferentes grandezas que são necessárias nos algoritmos de RL (e.g., retorno de episódios, gradientes de políticas, ...).

Esse é o objetivo principal do ciclo de interação com o ambiente: permitir ao agente explorar o ambiente e coletar dados para seu aprendizado.

> **Atenção**: Praticamente todos os pacotes de RL disponíveis implementam uma versão desse loop de interação. Dessa forma, independentemente se seu objetivo é desenvolver sua própria biblioteca de RL ou apenas re-utilizar código pré-existente, é importante entender os principais conceitos envolvidos!

Nessa seção testaremos a política aleatória em uma outra versão do ambiente MountainCar:

In [59]:
env = gym.make("MountainCarContinuous-v0")
agent = RandomPolicy(env.observation_space, env.action_space)

---

**<font color="red">EXERCÍCIO-PROGRAMA 1:</font>**

Nesse exercício você deverá utilizar os métodos da API do Gym a fim de permitir que um agente de RL simule um episódio. Complete a função `sample_episode` com seu código e preste atenção para retornar as variáveis definidas na documentação. Caso necessário revise a <a href="/lab#1.2-Interface-do-Gym:-métodos-reset(),-step()-e-render()" target="_self">Seção 1.2</a>.

>**Atenção**: Para visualizar o episódio não se esqueça de chamar `env.render()` durante a simulação da trajetória. Use a flag `render` em um *if-statement* do Python para dinamicamente habilitar a visualização. Uma vez que a parte gráfica consome muito tempo, é comum desabilitar a renderização do ambiente durante o treinamento e avaliação de um agente de RL. Além disso, chame `env.close()` ao final do ciclo para fechar a janela de simulação.

In [60]:
def sample_episode(agent, env, render=False):
    """
    Simula um episódio completo de interação do agente com o ambiente.
    
    Args:
        agent (RLAgent):       agente responsável por retornar ações.
        env (gym.Environment): simulador de ambiente do OpenAI Gym.
        render (bool):         (opcional) flag para habilitar a renderização do ambiente.
        
    Return:
        (total_reward, episode_length): retorno obtido pelo agente no episódio e
        número de passos de decisão realizados no episódio.
    """
    total_reward = 0.0
    episode_length = 0

    # YOUR CODE HERE =====================================
        
    done = False
    obs = env.reset()
    
    while episode_length < env.spec.max_episode_steps and not done:

        action = agent.act(obs)
        
        obs, reward, done, info = env.step(action)
        
        episode_length += 1
        total_reward += reward
        if render:
            env.render()
    
    env.close()
    
    # ===================================================

    return total_reward, episode_length


Execute o código abaixo para testar a sua implementação.

In [61]:
total_reward, episode_length = sample_episode(agent, env, render=True)
print(f"return = {total_reward:.4f}, passos de decisão = {episode_length}\r", end="")

A menos que você tenha tido sorte na simulação, você deve ter observado que o carro nunca chega próximo à linha de chegada. :(

Não se preocupe vamos resolver isso nas próximas aulas!

---

Uma vez implementado a função `sample_episode` você pode amostrar diferentes trajetórias com a função `run` definida abaixo:

In [62]:
def run(agent, env, num_episodes):
    episode_returns, episode_lengths = [], []

    for episode in range(num_episodes):
        total_reward, episode_length = sample_episode(agent, env)
    
        episode_returns.append(total_reward)
        episode_lengths.append(episode_length)

        if episode % 10 == 0:
            print(f"episode = {episode}, return = {total_reward:.4f}, length = {episode_length}\r", end="")

    return episode_returns, episode_lengths
    

Execute o código abaixo para simular `NUM_EPISODES` trajetórias:

In [ ]:
NUM_EPISODES = 200

episode_returns, episode_lengths = run(agent, env, NUM_EPISODES)

plot_episode_total_rewards(episode_returns)

> **Atenção**: se você obteve um pico de retorno com um valor positivo (i.e., *outlier*), execute novamente a simulação. Caso contrário, você deve ter obtido um retorno médio (vide linha vermelha) entre -34 e -33 para `NUM_EPISODES==200`.

---

**<font color="red">QUESTÕES:</font>**

1. Qual a diferença entre o ambiente `MountainCar-v0` utilizado como exemplo na <a href="/lab#1.-Ambientes-no-OpenAI-Gym" target="_self">Seção 1</a> e o ambiente `MountainCarContinuous-v0` que você acabou de simular?
2. Como você interpreta os gráficos acima `Episode Return` e `Episode Return (Histogram)` ? Como você explicaria essas variações ruidosas da recompensa total?
3. Se você executar a simulação várias vezes, obterá resultados ligeiramente diferentes? Ao que se deve essa incerteza nos resultados?
4. Você diria que o agente aleatório obteve uma boa performance? *Dica*: relacione os resultados obtidos com a especificação do ambiente `MountainCarContinuous-v0` (e.g., `env.spec`).
5. Note que durante a simulação o `Episode Length` se manteve constante ao longo dos episódios. O que isso significa do ponto de vista da tarefa de RL?